In [ ]:
from langchain_community.graphs import Neo4jGraph
# from neo4j.debug import watch
import pandas as pd
!pip install pyprojroot
from pyprojroot import here
!pip install neo4j
# watch("neo4j")

**Keep the following info as secret keys or in environment variables**

In [ ]:
NEO4J_URL = "bolt://localhost:7687"
NEO4J_USERNAME = "Enter user ID"
NEO4J_PASSWORD = "Enter password"
NEO4J_DATABASE = 'neo4j'

**Load an instance of the Graph DB**

In [ ]:
graph = Neo4jGraph(url=NEO4J_URL, username=NEO4J_USERNAME, password=NEO4J_PASSWORD, database=NEO4J_DATABASE)

**To load the data withing cypher query:**

--------------------
- NOTE: Uncomment `dbms.security.allow_csv_import_from_file_urls=true` in `neo4j.conf` to be able to load the file locally.
- NOTE: Absolute path + no space for the file directory.
--------------------

**Loading the data for test:**

In [ ]:
battery_csv_path = here("Enter your mastersheet")
print(pd.read_csv(battery_csv_path).columns)
print("DAta shape:", pd.read_csv(battery_csv_path).shape)

In [ ]:
pd.read_csv(battery_csv_path).head(3)

**Import movie information from the CSV file with tagline, construct the knowledge graph, and store the data in the Graph database**

In [ ]:
#Run this cell if you are modifying any connections or data of the BR-KG
graph.query("""
MATCH (n)
DETACH DELETE n
""")

In [ ]:

battery_csv_path = f"Enter your mastersheet"  # Add the file:// prefix

# Now use this path in your query
graph.query("""
LOAD CSV WITH HEADERS FROM $battery_recycling_directory AS row

// Create Battery_Type nodes
MERGE (b:Battery_Type {DOI: row.DOI})
SET b.abstract_number = row.`Abstract Number`,
    b.title = row.Title,
    b.abstract = row.Abstract,
    b.name = row.Battery_Type

// Create Recycling_Process_Used nodes and Processed_By relationships
MERGE (p:Recycling_Process_Used {DOI: row.DOI})
SET p.name = row.Recycling_Process_Used,
    p.total_cost = toFloat(row.`Total Cost Approx. ($)`),
    p.total_energy = toFloat(row.`Total Energy Approx. (J)`),
    p.abstract = row.Abstract
MERGE (b)-[:Processed_By]->(p)

// Create Recovered Materials nodes and Recovers relationships with efficiencies
FOREACH (material IN split(row.`Recovered Materials`, '|') |
    MERGE (rm:Recovered_Materials {name: trim(material)})
    FOREACH (efficiency IN split(row.Efficiencies, '|') |
        MERGE (p)-[:Recovers {efficiency: toFloat(trim(efficiency))}]->(rm))
)

// Create Process_Conditions/Battery_Properties nodes and Has_Conditions relationships
MERGE (c:Process_Conditions {details: trim(row.`Process_Conditions/Battery_Properties`)})
MERGE (p)-[:Has_Conditions]->(c)

// Create Reused relationships
FOREACH (material IN split(row.`Recovered Materials`, '|') |
    MERGE (rm:Recovered_Materials {name: trim(material)})
    MERGE (rm)-[:Reused]->(b))
""",
params={"battery_recycling_directory": battery_csv_path})

**Print the graph database schema and verify the knowledge graph**

In [ ]:
graph.refresh_schema()
print(graph.schema)